<a href="https://colab.research.google.com/github/trachtok/dspracticum2020_data/blob/main/assignment05/assignmen05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from google.colab import drive

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [10]:
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
!ls "/content/drive/My Drive/assignment05_data"

batman	superman


In [32]:
train_dir="/content/drive/My Drive/assignment05_data"

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

datagen = ImageDataGenerator(validation_split=0.2)

In [33]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation') # set as validation data

Found 468 images belonging to 2 classes.
Found 116 images belonging to 2 classes.
